In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('tmdb_movies_data.csv')


In [4]:
df = df[['id','original_title','cast','director','keywords','overview','genres','release_year','vote_average']]

In [5]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [6]:
df.drop(df[df['original_title'] == 'Cazuza - O tempo nÃ£o pÃ¡ra'].index, inplace= True)
df.drop(df[df['original_title'] == 'í•˜ìš¸ë§'].index, inplace= True)
df = df.reset_index(drop=True)

In [7]:
def remove(obj):
    return obj.split('|')

In [8]:
df['cast'] = df['cast'].apply(lambda x:remove(str(x)))

In [9]:
df['keywords'] = df['keywords'].apply(lambda x:remove(str(x)))
df['genres'] = df['genres'].apply(lambda x:remove(str(x)))


In [10]:
rec_df = df

In [11]:
df[df['id'] == 355338]

,id,original_title,cast,director,keywords,overview,genres,release_year,vote_average
251,355338,Riley's First Date?,"[Amy Poehler, Phyllis Smith, Bill Hader, Lewis...",Josh Cooley,"[mother daughter relationship, rock music, gir...","Riley, now 12, is hanging out at home with her...","[Animation, Family]",2015,7.3


In [12]:
rec_df['genres'] = rec_df['genres'].apply(lambda x:x[0])

In [13]:
rec_df = rec_df[['id','original_title','genres','vote_average']]

In [14]:
import pickle
pickle.dump(rec_df,open('movie_data.sav','wb'))

In [15]:
df['cast'] = df['cast'].apply(lambda x:x[:4])


In [16]:
df['cast'] = df['cast'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [17]:
df['director'] = df['director'].apply(lambda x:[str(x).replace(' ','')])

In [18]:
df['keywords'] = df['keywords'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [19]:
df['genres'] = df['genres'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [20]:
df['overview'] = df['overview'].apply(lambda x:str(x).split(' '))

In [21]:
df['release_year'] = df['release_year'].apply(lambda x:[str(x)])

In [22]:
df['corpus'] = df['overview'] + df['keywords'] + df['genres'] + df['cast'] + df['director'] + df['release_year']

In [23]:
new_df = df[['id','original_title','corpus','vote_average']]

In [24]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))

/tmp/ipykernel_3477/3919970262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))


In [25]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))

/tmp/ipykernel_3477/420603864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))


In [26]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [27]:
def stem(text):
    words = []
    for i in text.split():
        words.append(ps.stem(i))
    
    return " ".join(words)

In [28]:
# new_df['corpus'] = new_df['corpus'].apply(stem)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

# Stop Words = a,the,of,after,now

In [30]:
cv = CountVectorizer(stop_words='english',min_df=2)

In [31]:
data = cv.fit_transform(new_df['corpus']).toarray()

In [32]:
    df

,id,original_title,cast,director,keywords,overview,genres,release_year,vote_average,corpus
0,135397,Jurassic World,"[ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...",[ColinTrevorrow],"[monster, dna, tyrannosaurusrex, velociraptor,...","[Twenty-two, years, after, the, events, of, Ju...","[A, c, t, i, o, n]",[2015],6.5,"[Twenty-two, years, after, the, events, of, Ju..."
1,76341,Mad Max: Fury Road,"[TomHardy, CharlizeTheron, HughKeays-Byrne, Ni...",[GeorgeMiller],"[future, chase, post-apocalyptic, dystopia, au...","[An, apocalyptic, story, set, in, the, furthes...","[A, c, t, i, o, n]",[2015],7.1,"[An, apocalyptic, story, set, in, the, furthes..."
2,262500,Insurgent,"[ShaileneWoodley, TheoJames, KateWinslet, Anse...",[RobertSchwentke],"[basedonnovel, revolution, dystopia, sequel, d...","[Beatrice, Prior, must, confront, her, inner, ...","[A, d, v, e, n, t, u, r, e]",[2015],6.3,"[Beatrice, Prior, must, confront, her, inner, ..."
3,140607,Star Wars: The Force Awakens,"[HarrisonFord, MarkHamill, CarrieFisher, AdamD...",[J.J.Abrams],"[android, spaceship, jedi, spaceopera, 3d]","[Thirty, years, after, defeating, the, Galacti...","[A, c, t, i, o, n]",[2015],7.5,"[Thirty, years, after, defeating, the, Galacti..."
4,168259,Furious 7,"[VinDiesel, PaulWalker, JasonStatham, Michelle...",[JamesWan],"[carrace, speed, revenge, suspense, car]","[Deckard, Shaw, seeks, revenge, against, Domin...","[A, c, t, i, o, n]",[2015],7.3,"[Deckard, Shaw, seeks, revenge, against, Domin..."
...,...,...,...,...,...,...,...,...,...,...
9298,21,The Endless Summer,"[MichaelHynson, RobertAugust, Lord'TallyHo'Ble...",[BruceBrown],"[surfer, surfboard, surfing]","[The, Endless, Summer,, by, Bruce, Brown,, is,...","[D, o, c, u, m, e, n, t, a, r, y]",[1966],7.4,"[The, Endless, Summer,, by, Bruce, Brown,, is,..."
9299,20379,Grand Prix,"[JamesGarner, EvaMarieSaint, YvesMontand, Tosh...",[JohnFrankenheimer],"[carrace, racing, formula1]","[Grand, Prix, driver, Pete, Aron, is, fired, b...","[A, c, t, i, o, n]",[1966],5.7,"[Grand, Prix, driver, Pete, Aron, is, fired, b..."
9300,39768,Beregis Avtomobilya,"[InnokentiySmoktunovskiy, OlegEfremov, GeorgiZ...",[EldarRyazanov],"[car, trolley, stealingcar]","[An, insurance, agent, who, moonlights, as, a,...","[M, y, s, t, e, r, y]",[1966],6.5,"[An, insurance, agent, who, moonlights, as, a,..."
9301,21449,"What's Up, Tiger Lily?","[TatsuyaMihashi, AkikoWakabayashi, MieHama, Jo...",[WoodyAllen],[spoof],"[In, comic, Woody, Allen's, film, debut,, he, ...","[A, c, t, i, o, n]",[1966],5.4,"[In, comic, Woody, Allen's, film, debut,, he, ..."


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
similarity = cosine_similarity(data)

In [35]:
new_df = new_df.rename({'original_title':'title'},axis=1)

In [36]:
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:11]

[(4395, 0.27931019386546413),
 (8695, 0.23911404992940524),
 (8666, 0.22156468376279895),
 (5701, 0.18181818181818185),
 (405, 0.16283473681973243),
 (37, 0.15974461276617435),
 (226, 0.14083575804390608),
 (6635, 0.1371450962647483),
 (7723, 0.13483997249264842),
 (85, 0.13273296830047474)]

In [37]:
df.iloc[0]

id                                                           135397
original_title                                       Jurassic World
cast              [ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...
director                                           [ColinTrevorrow]
keywords          [monster, dna, tyrannosaurusrex, velociraptor,...
overview          [Twenty-two, years, after, the, events, of, Ju...
genres                                           [A, c, t, i, o, n]
release_year                                                 [2015]
vote_average                                                    6.5
corpus            [Twenty-two, years, after, the, events, of, Ju...
Name: 0, dtype: object

In [38]:
def recommend(movie):
    movie_index = new_df[new_df['id'] == int(movie)].index[0]
    movie_list = sorted(list(enumerate(similarity[movie_index])),reverse=True,key= lambda x:x[1])[1:11]
    rec_movies = []
    for i in movie_list:
        rec_movies.append(new_df.iloc[i[0]][0])
    return {'ids':rec_movies}

In [39]:
new_df[new_df['id'] == 253412].index[0]

52

In [60]:
new_df[new_df['title'] == 'The Dark Knight'].iloc[0][0]

155

In [ ]:
recommend(76341)


{'ids': [27786, 9659, 14642, 20504, 38695, 11036, 4917, 310135, 287424, 84226]}

In [ ]:
genre = [
    {
        "id": 28,
        "name": "Action"
    },
    {
        "id": 12,
        "name": "Adventure"
    },
    {
        "id": 16,
        "name": "Animation"
    },
    {
        "id": 35,
        "name": "Comedy"
    },
    {
        "id": 80,
        "name": "Crime"
    },
    {
        "id": 99,
        "name": "Documentary"
    },
    {
        "id": 18,
        "name": "Drama"
    },
    {
        "id": 10751,
        "name": "Family"
    },
    {
        "id": 14,
        "name": "Fantasy"
    },
    {
        "id": 36,
        "name": "History"
    },
    {
        "id": 27,
        "name": "Horror"
    },
    {
        "id": 10402,
        "name": "Music"
    },
    {
        "id": 9648,
        "name": "Mystery"
    },
    {
        "id": 10749,
        "name": "Romance"
    },
    {
        "id": 878,
        "name": "Science Fiction"
    },
    {
        "id": 10770,
        "name": "TV Movie"
    },
    {
        "id": 53,
        "name": "Thriller"
    },
    {
        "id": 10752,
        "name": "War"
    },
    {
        "id": 37,
        "name": "Western"
    }
]

In [ ]:
import json
movie_lists = []
for i in genre:
    movie_lists.append({ 
        i['name'] :rec_df[rec_df['genres'] == i['name']].iloc[0:19]['id'].to_list()
    })


In [ ]:
new_df.reset_index(drop=True)
pickle.dump(new_df,open('data.sav','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.sav','wb'))

In [ ]:
import random
print(random.randint(0,9306))

4538
